In [1]:
# Import libraries
import os
import numpy as np
import pandas as pd
import requests
import re

In [2]:
# Web scrapping using pandas

# Light-Duty Vehicle Registration Counts by State and Fuel Type
url = 'https://afdc.energy.gov/vehicle-registration?year='

vRegistration_years = [2016, 2017, 2018, 2019, 2020, 2021, 2022]

df_registrations = pd.DataFrame()
for n in vRegistration_years:
    this_url = url + f'{n}'
    print(this_url)  
    this_df = pd.read_html(this_url, header = 1)[0]
    # remove total
    this_df = this_df.iloc[:-1]
    # add year
    this_df.insert(0, "Year", [n] * this_df.shape[0])

    df_registrations = pd.concat([df_registrations, this_df])

# Save data frame to csv file
data_dir = "data"
registrations_csv_file = 'data_ld_vregistration_by_state_by_year.csv'
registrations_csv_filepath = os.path.join(data_dir, registrations_csv_file)
df_registrations.to_csv(registrations_csv_filepath, index = False)

https://afdc.energy.gov/vehicle-registration?year=2016
https://afdc.energy.gov/vehicle-registration?year=2017
https://afdc.energy.gov/vehicle-registration?year=2018
https://afdc.energy.gov/vehicle-registration?year=2019
https://afdc.energy.gov/vehicle-registration?year=2020
https://afdc.energy.gov/vehicle-registration?year=2021
https://afdc.energy.gov/vehicle-registration?year=2022


In [3]:
print(registrations_csv_file)
df_registrations.head()

data_ld_vregistration_by_state_by_year.csv


,Year,State,Electric (EV),Plug-In Hybrid Electric (PHEV),Hybrid Electric (HEV),Biodiesel,Ethanol/Flex (E85),Compressed Natural Gas (CNG),Propane,Hydrogen,Methanol,Gasoline,Diesel,Unknown Fuel
0,2016,Alabama,500,900,29100,0,428300,20100,0,0,0,3777300,126500,53900
1,2016,Alaska,200,200,5000,0,55700,4900,0,0,0,525900,44800,19400
2,2016,Arizona,4700,4400,89600,0,427300,17500,0,0,100,4805000,179500,112800
3,2016,Arkansas,200,500,19100,0,320500,12600,0,0,0,2097800,96800,22200
4,2016,California,141500,116700,966700,0,1322600,80600,0,1300,400,27241000,710400,115500


In [4]:
def incentive_str_to_num(string):
    # find individual numbers
    str_num = re.findall(r'[\d]+[.,\d]+|[\d]+', string)
    # convert to float
    num = [float(s.replace(',','')) for s in str_num]
    # take maximum of all numbers, or 0 if no number
    incentive = max(num) if len(num) > 0 else 0
    # print(string, ' : converted to : ', num, ' : then to : ', incentive)
    return incentive

## Web scrapping using pandas
# State Electric Vehicle Tax Credits and Registration Fees, in US Dollars (July 2023)
url_taxfoundation = 'https://taxfoundation.org/data/all/state/electric-vehicles-ev-taxes-state'

years = np.arange(2016,2022+1)

df = pd.read_html(url_taxfoundation, header = 0)[0]
df['Incentive ($)'] = df.apply(lambda row: incentive_str_to_num(row['EV Purchase Tax Credit']), axis=1)
df['Additional Registration Fee ($/year)'] = df.apply(lambda row: incentive_str_to_num(row['Additional EV Annual Registration Fee']), axis=1)

df_incentives = pd.DataFrame()
for y in years:
    this_df = df.assign(Year=[y] * df.shape[0])
    df_incentives = pd.concat([df_incentives, this_df])

# Wikipedia incentives overwrite Tax Foundation incentives (where available)    
data_dir = "data"
wikipedia_file = 'incentives_wikipedia_v2.csv'
wikipedia_filepath = os.path.join(data_dir, wikipedia_file)

df_wikipedia = pd.read_csv(wikipedia_filepath)
df_incentives = pd.merge(df_incentives, df_wikipedia, on=['State', 'Year'], how='left')
df_incentives['Incentive ($)'] = df_incentives['Incentive Wikipedia ($)'].fillna(df_incentives['Incentive ($)'])

## Apply manual correction
data_dir = "data"
manual_corrections_file = 'ev_credits_manual_corrections.csv'
manual_corrections_filepath = os.path.join(data_dir, manual_corrections_file)

df_correction = pd.read_csv(manual_corrections_filepath)
# convert to stacked format
df_correction_stacked = pd.melt(
    df_correction,
    id_vars=['State'],
    value_vars=['2016','2017','2018','2019','2020','2021','2022'],
    var_name='Year',
    value_name='Incentive - correction',
)
df_correction_stacked.Year = pd.to_numeric(df_correction_stacked.Year, errors='coerce')
# apply correction
df_incentives = pd.merge(df_incentives, df_correction_stacked, on=['State', 'Year']) # TODO: inner? left?
df_incentives['Incentive ($)'] = df_incentives['Incentive - correction'].fillna(df_incentives['Incentive ($)'])

# Save data frame to csv file
incentives_csv_file = 'data_ev_credits_fees_by_state_by_year_v4.csv'
incentives_csv_filepath = os.path.join(data_dir, incentives_csv_file)
df_incentives.to_csv(incentives_csv_filepath, index = False)

In [5]:
print(incentives_csv_file)
df_incentives.head()

data_ev_credits_fees_by_state_by_year_v4.csv


,State,EV Purchase Tax Credit,Additional EV Annual Registration Fee,Additional Hybrid Annual Registration Fee,Incentive ($),Additional Registration Fee ($/year),Year,Amount of incentive,Type of PEV/vehicle,Type of incentive,Carpool lane access,Comments,EV_purchase_credit,Incentive Wikipedia ($),Incentive - correction
0,Alabama,0,200,100,0.0,200.0,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alaska,1000,0,0,0.0,0.0,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,Arizona,0,0,0,0.0,0.0,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arkansas,0,200,50-100 (a),0.0,200.0,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,California,"750-7,500",108,0,2000.0,108.0,2016,"up to $2,500",BEVs,Purchase rebate,Yes,"Free access to HOVs through January 1, 2019, w...",True,2500.0,2000.0


In [6]:
# Add column with political leaning: dem_margin is the difference between 
# democrat vote share and the republican vote share (presidential elections 2020)
# load data from https://www.cookpolitical.com/2020-national-popular-vote-tracker
data_dir = "data"
pres_elections_2020_file = 'Popular vote backend - Sheet1.csv'
pres_elections_2020_filepath = os.path.join(data_dir, pres_elections_2020_file)

df_2020_pres_elections = pd.read_csv(pres_elections_2020_filepath)
# clean up table formatting
df_2020_pres_elections = df_2020_pres_elections.drop(df_2020_pres_elections.index[[0,1,2,3,8,11,19,27,36,43,44]])

df_2020_dem_margin = df_2020_pres_elections[['state', 'dem_this_margin']]
df_2020_dem_margin = df_2020_dem_margin.rename(columns={'state': 'State', 'dem_this_margin': 'dem_margin'})
df_2020_dem_margin.loc[:, 'dem_margin'] = df_2020_dem_margin['dem_margin'].str.rstrip('%').astype('float')

df_political_leaning = pd.DataFrame()
for y in range(2016,2022+1):
    this_df = df_2020_dem_margin.assign(Year=[y] * df_2020_dem_margin.shape[0])
    df_political_leaning = pd.concat([df_political_leaning, this_df])

# Save data frame to csv file
political_leaning_csv_file = 'data_political_leaning.csv'
political_leaning_csv_filepath = os.path.join(data_dir, political_leaning_csv_file)
df_political_leaning.to_csv(political_leaning_csv_filepath, index = False)

In [7]:
print(political_leaning_csv_file)
df_political_leaning.head()

data_political_leaning.csv


,State,dem_margin,Year
4,Arizona,0.3,2016
5,Florida,-3.4,2016
6,Georgia,0.2,2016
7,Iowa,-8.2,2016
9,Michigan,2.8,2016


In [8]:
# Add column with number of charging stations and number of charging outlets
# load data from https://afdc.energy.gov/files/docs/historical-station-counts.xlsx?year=2023
data_dir = "data"
stations_file = 'historical-station-counts.xlsx'
stations_filepath = os.path.join(data_dir, stations_file)

df_stations = pd.DataFrame()

for year in range(2016, 2023):
    
    df = pd.read_excel(stations_filepath, sheet_name=str(year), header=1)
    # clean up rows
    df.dropna(subset=['State', 'Electrica'], inplace=True)
    df = df.iloc[:-1]

    # convert string column to two numeric columns
    df[['num_stations', 'num_charging_outlets']] = df['Electrica'].str.split('|', n=1, expand=True)
    df['num_stations'] = pd.to_numeric(df['num_stations'].str.replace(',',''))
    df['num_charging_outlets'] = pd.to_numeric(df['num_charging_outlets'].str.replace(',',''))

    # Add column for year
    this_df = df.assign(Year=[year] * df.shape[0])

    df_stations = pd.concat([df_stations, this_df])

    # Save data frame to csv file
    stations_csv_file = 'data_ev_charging_stations.csv'
    stations_csv_filepath = os.path.join(data_dir, stations_csv_file)
    df_stations.to_csv(stations_csv_filepath, index = False)

In [9]:
print(stations_csv_file)
df_stations.head()

data_ev_charging_stations.csv


,State,Biodiesel,CNG,E85,Electrica,Hydrogenb,LNG,Propanec,Renewable Diesel,Totald,num_stations,num_charging_outlets,Year
2,Alabama,5.0,32.0,32.0,113 | 227,0 | 0 | 0,2.0,2 | 90 | 92,0.0,390.0,113,227,2016
4,Alaska,0.0,1.0,0.0,3 | 5,0 | 0 | 0,0.0,0 | 7 | 7,0.0,13.0,3,5,2016
6,Arizona,76.0,36.0,25.0,424 | 988,0 | 0 | 0,8.0,15 | 78 | 93,0.0,1226.0,424,988,2016
8,Arkansas,5.0,17.0,44.0,63 | 89,0 | 0 | 0,1.0,1 | 44 | 45,0.0,201.0,63,89,2016
10,California,42.0,319.0,119.0,"4,184 | 13,655",25 | 14 | 39,44.0,34 | 288 | 322,0.0,14540.0,4184,13655,2016
